<a href="https://colab.research.google.com/github/kip-07/GooglecolabAI/blob/main/Best_First_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import heapq
from IPython.display import HTML, display


In [36]:
def display_colored_grid(grid):
    html = '<table style="border-collapse: collapse;">'
    for row in grid:
        html += '<tr>'
        for cell in row:
            color = 'white'
            text_color = 'black'  # Ensure numbers are black
            display_value = cell if cell != 99 else ''
            font_size = '15px' if cell != 99 else '0px'
            if cell == 99:
                color = 'black'
            html += f'<td style="width: 20px; height: 20px; background-color: {color}; border: 1px solid gray; text-align: center; vertical-align: middle; font-family: Arial; font-size: {font_size}; color: {text_color};">{display_value}</td>'
        html += '</tr>'
    html += '</table>'
    display(HTML(html))

In [27]:
# Section 3: Maze Generation
def generate_static_maze():
    return [
        [0, 0, 0, 0, 99, 0, 0, 0, 99, 0, 0, 0, 0, 0, 0],
        [0, 99, 99, 0, 99, 0, 99, 0, 99, 0, 99, 99, 99, 99, 0],
        [0, 0, 0, 0, 0, 0, 99, 0, 0, 0, 99, 0, 0, 0, 0],
        [99, 99, 99, 99, 99, 0, 99, 99, 99, 0, 99, 0, 99, 99, 0],
        [0, 0, 0, 0, 99, 0, 0, 0, 99, 0, 0, 0, 0, 99, 0],
        [0, 99, 99, 0, 99, 99, 99, 0, 99, 99, 99, 99, 0, 99, 0],
        [0, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99, 0],
        [0, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 0, 99, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99, 0, 0, 0],
        [0, 99, 99, 99, 0, 99, 99, 99, 99, 99, 0, 99, 99, 99, 0],
        [0, 0, 0, 99, 0, 0, 0, 0, 0, 99, 0, 0, 0, 99, 0],
        [99, 99, 0, 99, 99, 99, 99, 99, 0, 99, 99, 99, 0, 99, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99, 0, 99, 0],
        [0, 99, 99, 99, 99, 99, 99, 99, 99, 99, 0, 99, 0, 99, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]

In [21]:
# Section 4: Best-First Search Algorithm
def manhattan_distance(a, b):
    """Computes the Manhattan Distance between two points."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def best_first_search(grid, start, goal):
    """Performs Best-First Search to find the treasure in the grid."""
    rows, cols = len(grid), len(grid[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    pq = [(manhattan_distance(start, goal), start)]
    visited = set()

    while pq:
        _, current = heapq.heappop(pq)
        if current in visited:
            continue
        visited.add(current)

        x, y = current
        grid[x][y] = manhattan_distance(current, goal)  # Store heuristic value

        if current == goal:
            return  # Treasure found

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and (nx, ny) not in visited and grid[nx][ny] != 99:
                heapq.heappush(pq, (manhattan_distance((nx, ny), goal), (nx, ny)))


In [28]:
def run_treasure_hunt():
    grid = generate_static_maze()
    start, goal = (0, 0), (14, 14)
    best_first_search(grid, start, goal)
    display_colored_grid(grid)

In [29]:
run_treasure_hunt()

28,27,26,25,,0,0,0,,0,0,0,0,0,0
0,,,24,,0,,0,,0,,,,,0
0,0,0,23,22,21,,0,0,0,,0,0,0,0
,,,,,20,,,,0,,0,,,0
0,0,0,0,,19,18,17,,0,0,0,0,,0
0,,,0,,,,16,,,,,0,,0
0,,0,0,0,0,0,15,14,13,12,11,10,,0
0,,,,,,,,,,,,9,,0
0,0,0,0,0,0,0,0,0,0,0,,8,7,6
0,,,,0,,,,,,0,,,,5
0,0,0,,0,0,0,0,0,,0,0,0,,4


In [64]:
#8 PUZZLE PROBLEM
# Define goal state
GOAL_STATE = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
MOVES = {"UP": (-1, 0), "DOWN": (1, 0), "LEFT": (0, -1), "RIGHT": (0, 1)}


In [38]:
def display_colored_grid(grid):
    html = '<table style="border-collapse: collapse;">'
    for row in grid:
        html += '<tr>'
        for cell in row:
            html += f'''<td style="width: 30px; height: 30px; background-color: white;
                         border: 1px solid gray; text-align: center; vertical-align: middle;
                         font-family: Arial; font-size: 12px;">{cell}</td>'''
        html += '</tr>'
    html += '</table>'
    display(HTML(html))

In [56]:
def validate_cell(cell):
    x, y = cell
    return 0 <= x <= 2 and 0 <= y <= 2

def randomize_grid(grid, zero, moves=20):
    directions = list(MOVES.values())
    for _ in range(moves):
        random.shuffle(directions)
        for direction in directions:
            new_cell = (zero[0] + direction[0], zero[1] + direction[1])
            if validate_cell(new_cell):
                x, y = zero
                new_x, new_y = new_cell
                grid[x][y], grid[new_x][new_y] = grid[new_x][new_y], grid[x][y]
                zero = new_cell
                break
    return zero


In [68]:
class PuzzleState:
    def __init__(self, grid, zero_pos, g, parent=None):
        self.grid = grid
        self.zero_pos = zero_pos
        self.g = g
        self.parent = parent

    def __lt__(self, other):
        return (self.g + self.heuristic()) < (other.g + other.heuristic())

    def heuristic(self):
        distance = 0
        for r in range(3):
            for c in range(3):
                value = self.grid[r][c]
                if value != 0:
                    goal_r, goal_c = divmod(value - 1, 3)
                    distance += abs(goal_r - r) + abs(goal_c - c)
        return distance

    def is_goal(self):
        return self.grid == GOAL_STATE

    def get_next_states(self):
        next_states = []
        x, y = self.zero_pos
        for move, (dx, dy) in MOVES.items():
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_grid = [row[:] for row in self.grid]
                new_grid[x][y], new_grid[new_x][new_y] = new_grid[new_x][new_y], new_grid[x][y]
                next_states.append(PuzzleState(new_grid, (new_x, new_y), self.g + 1, self))
        return next_states


In [45]:
def solve_puzzle(initial_grid):
    zero_pos = next((r, c) for r in range(3) for c in range(3) if initial_grid[r][c] == 0)
    start_state = PuzzleState(initial_grid, zero_pos, 0)
    open_set = []
    heapq.heappush(open_set, start_state)
    visited = set()
    visited.add(tuple(map(tuple, start_state.grid)))

    while open_set:
        current = heapq.heappop(open_set)
        if current.is_goal():
            path = []
            while current:
                path.append(current.grid)
                current = current.parent
            return path[::-1]

        for next_state in current.get_next_states():
            grid_tuple = tuple(map(tuple, next_state.grid))
            if grid_tuple not in visited:
                heapq.heappush(open_set, next_state)
                visited.add(grid_tuple)
    return None


In [58]:
# Example: Initial state
grid = [[1, 2, 3], [4, 0, 5], [6, 7, 8]]
zero = (1, 1)
zero = randomize_grid(grid, zero)
display_colored_grid(grid)


1,2,3
4,0,7
6,8,5


In [69]:
# Solve the puzzle
solution_path = solve_puzzle(grid)
if solution_path:
    for step, state in enumerate(solution_path):
        print(f"Step {step}:")
        display_colored_grid(state)
        print("\n")
else:
    print("No solution found!")


Step 0:


1,2,3
4,0,7
6,8,5




Step 1:


1,2,3
4,8,7
6,0,5




Step 2:


...

1,2,3
4,0,5
7,8,6




Step 13:


1,2,3
4,5,0
7,8,6




Step 14:


1,2,3
4,5,6
7,8,0
